In [3]:
import iris
import glob
import iris.plot as iplt
import iris.quickplot as qplt
import datetime as datetime
import iris.coord_categorisation as cat
import numpy as np
import os
import sys

# surrogate = iris.load("/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20190307.nc")
# surrogate = surrogate[0]

# if 'projection_y_coordinate' not in surrogate.coords():
#     # Copy projection_y_coordinate from the first cube
#     day_cube.add_dim_coord(surrogate.coord('projection_y_coordinate'), 1)

# if 'projection_x_coordinate' not in surrogate.coords():
#     # Copy projection_x_coordinate from the first cube
#     day_cube.add_dim_coord(surrogate.coord('projection_x_coordinate'), 2)

# if 'time' not in surrogate.coords():
#     # Copy projection_x_coordinate from the first cube
#     day_cube.add_dim_coord(surrogate.coord('time'), 0)    


def concatenate_with_error_handling(cube_list):
    """Handle errors during cube concatenation by identifying problematic cubes."""
    problematic_cube_index = None
    start = 0

    for i, cube in enumerate(cube_list):
        try:
            cube_list[start:i + 1].concatenate_cube()
        except Exception as e:
            print(f"Error concatenating cube {i}: {str(e)}")
            problematic_cube_index = i
            start = i

    if 0 <= problematic_cube_index < len(cube_list):
        del cube_list[problematic_cube_index]
        print(f"Cube at index {problematic_cube_index} removed from CubeList.")
    else:
        print(f"Index {problematic_cube_index} is out of range for CubeList.")

    return cube_list.concatenate_cube()


def process_half_hour(cube, label):
    """Process half-hourly data, apply filters, and calculate statistics."""
    
    if cube is None:
        print(f"No values in {label}.")
        return None

    if len(cube.shape) == 2:
        print(f"Only 1 value in {label}.")
        return None

    if cube.shape[0] < 4:
        print(f"Only {cube.shape[0]} values in {label}.")
        return None

    # Append unfiltered data
    mean_cube = cube.copy().aggregated_by(['hour'], iris.analysis.MEAN)
    return mean_cube


def process_hour(cube):
    """Process half-hourly data, apply filters, and calculate statistics."""
    
    if cube is None:
        print(f"No values.")
        return None

    if len(cube.shape) == 2:
        print(f"Only 1 value.")
        return None

    if cube.shape[0] < 8:
        print(f"Only {cube.shape[0]} values in cube.")
        return None

    # Append unfiltered data
    mean_cube = cube.copy().aggregated_by(['hour'], iris.analysis.MEAN)
    return mean_cube


def create_year_directory(base_path, year):
    """Create directories for each year in the range if they don't exist."""
    for label in ['unfiltered']:
        year_path = os.path.join(base_path.format(label), str(year))
        os.makedirs(year_path, exist_ok=True)
        print(f"Directory created: {year_path}")


def process_and_save_cubes(cube_list, label, base_fp, year, iteration):
    """Concatenate cubes, filter data, and save to disk."""
    # Check if the cube list is empty
    if not cube_list:
        print(f"No cubes to process for {label}, year {year}, iteration {iteration}.")
        return

    # Ensure data in each cube is in the correct format
    for halfhour_i in range(len(cube_list)):
        cube_list[halfhour_i].data = cube_list[halfhour_i].data.astype('float64')

    try:
        # Attempt to concatenate cubes
        full_day_cube = cube_list.concatenate_cube()
    except Exception as e:
        print(f"Concatenation failed: {e}. Attempting error-handling concatenation.")
        full_day_cube = concatenate_with_error_handling(cube_list)

    # Print the maximum value in the cube's data
    print(f"Max value before saving: {np.nanmax(full_day_cube.data)}")
    print(full_day_cube.shape)

    # Save the concatenated cube to the specified file path
    save_path = base_fp.replace('unfiltered', label)
    iris.save(full_day_cube, save_path)

    # Print statistics for the cube
    print(f"Saved {label} cube for {year}, iteration {iteration}")
    print(f"Min: {np.nanmin(full_day_cube.data)}, Max: {np.nanmax(full_day_cube.data)}, Mean: {np.nanmean(full_day_cube.data)}")
    print(full_day_cube.shape)

def process_year(year):
    print(f"Processing year: {year}")

    radardir = f'/nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/{year}/'
    sorted_files = sorted(glob.glob(radardir + "*.nc"))

    for i, file_path in enumerate(sorted_files):
        print(f"Processing file {i + 1}/{len(sorted_files)}: {file_path}")

        new_fp_base = file_path.replace('OriginalFormat_1km/', 'OriginalFormat_1km/unfiltered/')
        new_fp_base = new_fp_base.replace('5mins', '1hr')[:-3] + '_1h.nc'
        new_fp_base = new_fp_base.replace('/2006/', f'/{year}/')

        if all(os.path.exists(new_fp_base.replace('unfiltered', label)) for label in ['unfiltered']):
            print("All files exist, skipping.")
            continue

        day_cube = iris.load_cube(file_path)
        if len(day_cube.shape)==2:
            day_cube = iris.util.new_axis(day_cube, "time")
        day_cube = day_cube[:,620:1800,210:1075]
        try:
            cat.add_hour(day_cube, 'time', name='hour')
        except:
            continue
        unfiltered_ls = iris.cube.CubeList()

        hours = set(day_cube.coord('hour').points)

        for hour in hours:
            hour_constraint = iris.Constraint(time=lambda cell: cell.point.hour == hour)
            hour_cube = day_cube.extract(hour_constraint)

            ##### SET negative values to NAN
            if np.nanmin(hour_cube.data) < 0:
                print(f"{hour}, min b4 correction", np.nanmin(hour_cube.data))
                # Ensure hour_cube.data is a MaskedArray
                hour_cube.data = np.ma.masked_where(hour_cube.data.mask, hour_cube.data)  # Safeguard if not already masked

                # Apply the condition only where the mask is False
                hour_cube.data = np.ma.masked_where(hour_cube.data.mask, np.where((hour_cube.data < 0) & (~hour_cube.data.mask), np.nan, hour_cube.data))
                print(f"{hour}, min after correction", np.nanmin(hour_cube.data))

            ########### Unfiltered version
            # Split the 5 minute cube into the first half hour and second half hour
            hour_cube_unfiltered = hour_cube.copy()

            # Get a first half and second half of hour aggregated to 30 mins
            hour_cube = process_hour(hour_cube_unfiltered)

            #  Add both to the lsit
            if hour_cube is not None:
                #  Add both to the lsit
                unfiltered_ls.append(hour_cube)


        for label, cube_list in [('unfiltered', unfiltered_ls)]:
            print(len(cube_list))
            process_and_save_cubes(cube_list, label, new_fp_base, year, i)  

for year in range(2006, 2020):
    base_path = '/nfs/a161/gy17m2a/PhD/datadir/NIMROD/1hr/OriginalFormat_1km/{}'
    create_year_directory(base_path, year)
    process_year(year)


Directory created: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/1hr/OriginalFormat_1km/unfiltered/2006
Processing year: 2006
Processing file 1/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060101.nc
All files exist, skipping.
Processing file 2/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060102.nc
All files exist, skipping.
Processing file 3/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060103.nc
All files exist, skipping.
Processing file 4/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060104.nc
All files exist, skipping.
Processing file 5/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2006/metoffice-c-band-rain-radar_uk_20060105.nc
All files exist, skipping.
Processing file 6/352: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1k

All files exist, skipping.
Processing file 14/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070114.nc
All files exist, skipping.
Processing file 15/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070115.nc
All files exist, skipping.
Processing file 16/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070116.nc
All files exist, skipping.
Processing file 17/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070117.nc
All files exist, skipping.
Processing file 18/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070118.nc
All files exist, skipping.
Processing file 19/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20070119.nc
All files exist, skipping.
Process

All files exist, skipping.
Processing file 347/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071215.nc
All files exist, skipping.
Processing file 348/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071216.nc
All files exist, skipping.
Processing file 349/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071217.nc
All files exist, skipping.
Processing file 350/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071218.nc
All files exist, skipping.
Processing file 351/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071219.nc
All files exist, skipping.
Processing file 352/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2007/metoffice-c-band-rain-radar_uk_20071220.nc
All files exist, skipping.
P

All files exist, skipping.
Processing file 174/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080625.nc
All files exist, skipping.
Processing file 175/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080626.nc
All files exist, skipping.
Processing file 176/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080627.nc
All files exist, skipping.
Processing file 177/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080628.nc
All files exist, skipping.
Processing file 178/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080629.nc
All files exist, skipping.
Processing file 179/362: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2008/metoffice-c-band-rain-radar_uk_20080630.nc
All files exist, skipping.
P

Only 1 value.
0
No cubes to process for unfiltered, year 2009, iteration 66.
Processing file 68/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar_uk_20090309.nc
All files exist, skipping.
Processing file 69/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar_uk_20090310.nc
All files exist, skipping.
Processing file 70/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar_uk_20090311.nc
All files exist, skipping.
Processing file 71/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar_uk_20090312.nc
All files exist, skipping.
Processing file 72/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar_uk_20090313.nc
All files exist, skipping.
Processing file 73/354: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2009/metoffice-c-band-rain-radar

...

Only 5 values in cube.
0
No cubes to process for unfiltered, year 2015, iteration 291.
Processing file 293/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2015/metoffice-c-band-rain-radar_uk_20151020.nc
Only 5 values in cube.
0
No cubes to process for unfiltered, year 2015, iteration 292.
Processing file 294/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2015/metoffice-c-band-rain-radar_uk_20151021.nc
Only 3 values in cube.
0
No cubes to process for unfiltered, year 2015, iteration 293.
Processing file 295/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2015/metoffice-c-band-rain-radar_uk_20151022.nc
Only 3 values in cube.
0
No cubes to process for unfiltered, year 2015, iteration 294.
Processing file 296/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2015/metoffice-c-band-rain-radar_uk_20151023.nc
Only 7 values in cube.
0
No cubes to process for unfiltered, year 2015, iteration 295.
Processing file 297/365: 

Only 1 value.
0
No cubes to process for unfiltered, year 2016, iteration 298.
Processing file 300/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rain-radar_uk_20161026.nc
All files exist, skipping.
Processing file 301/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rain-radar_uk_20161027.nc
All files exist, skipping.
Processing file 302/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rain-radar_uk_20161028.nc
All files exist, skipping.
Processing file 303/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rain-radar_uk_20161029.nc
All files exist, skipping.
Processing file 304/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rain-radar_uk_20161030.nc
All files exist, skipping.
Processing file 305/366: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2016/metoffice-c-band-rai

All files exist, skipping.
Processing file 314/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171110.nc
All files exist, skipping.
Processing file 315/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171111.nc
All files exist, skipping.
Processing file 316/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171112.nc
All files exist, skipping.
Processing file 317/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171113.nc
All files exist, skipping.
Processing file 318/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171114.nc
All files exist, skipping.
Processing file 319/365: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2017/metoffice-c-band-rain-radar_uk_20171115.nc
All files exist, skipping.
P

All files exist, skipping.
Processing file 348/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181216.nc
All files exist, skipping.
Processing file 349/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181217.nc
All files exist, skipping.
Processing file 350/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181218.nc
All files exist, skipping.
Processing file 351/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181219.nc
All files exist, skipping.
Processing file 352/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181220.nc
All files exist, skipping.
Processing file 353/363: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2018/metoffice-c-band-rain-radar_uk_20181221.nc
All files exist, skipping.
P

All files exist, skipping.
Processing file 282/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191010.nc
All files exist, skipping.
Processing file 283/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191011.nc
All files exist, skipping.
Processing file 284/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191012.nc
All files exist, skipping.
Processing file 285/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191013.nc
All files exist, skipping.
Processing file 286/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191014.nc
All files exist, skipping.
Processing file 287/364: /nfs/a161/gy17m2a/PhD/datadir/NIMROD/5mins/OriginalFormat_1km/2019/metoffice-c-band-rain-radar_uk_20191015.nc
All files exist, skipping.
P